Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

Logistic regression 
--------

In [5]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits)) + 0.005*tf.nn.l2_loss(weights)
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.

    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 34.206039
Training accuracy: 11.5%
Validation accuracy: 14.7%
Loss at step 100: 10.518215
Training accuracy: 72.3%
Validation accuracy: 71.6%
Loss at step 200: 6.247088
Training accuracy: 75.8%
Validation accuracy: 74.7%
Loss at step 300: 3.834617
Training accuracy: 78.4%
Validation accuracy: 76.6%
Loss at step 400: 2.463295
Training accuracy: 80.5%
Validation accuracy: 78.5%
Loss at step 500: 1.685096
Training accuracy: 82.4%
Validation accuracy: 79.8%
Loss at step 600: 1.241890
Training accuracy: 83.5%
Validation accuracy: 80.9%
Loss at step 700: 0.987889
Training accuracy: 84.1%
Validation accuracy: 81.5%
Loss at step 800: 0.841418
Training accuracy: 84.6%
Validation accuracy: 82.2%
Test accuracy: 87.9%


Neural network
--------

#### light version 


In [7]:
batch_size = 128
hidden_layers_size = 1024

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(tf.truncated_normal([hidden_layers_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layers_size]))
    hidden_biases = tf.Variable(tf.truncated_normal([hidden_layers_size]))
    
    logit_hidden = tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases
    
    logit = tf.matmul(tf.nn.relu(logit_hidden), weights) + biases

    # Training computation.
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logit)) \
            + 0.01*tf.nn.l2_loss(weights) + 0.01*tf.nn.l2_loss(hidden_weights) 

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logit)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases), weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases), weights) + biases)


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 3429.751465
Minibatch accuracy: 4.7%
Validation accuracy: 29.3%
Minibatch loss at step 500: 1156.978760
Minibatch accuracy: 80.5%
Validation accuracy: 79.5%
Minibatch loss at step 1000: 422.824860
Minibatch accuracy: 82.8%
Validation accuracy: 82.2%
Minibatch loss at step 1500: 155.398193
Minibatch accuracy: 78.1%
Validation accuracy: 80.9%
Minibatch loss at step 2000: 57.232025
Minibatch accuracy: 90.6%
Validation accuracy: 85.7%
Minibatch loss at step 2500: 21.540186
Minibatch accuracy: 83.6%
Validation accuracy: 85.4%
Minibatch loss at step 3000: 8.355812
Minibatch accuracy: 85.2%
Validation accuracy: 85.0%
Test accuracy: 90.8%


#### heavy version 

In [9]:
batch_size = 128
hidden_layers_size = 1024

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    input_weights = tf.Variable(tf.truncated_normal([image_size * image_size, batch_size]))
    input_biases = tf.Variable(tf.truncated_normal([batch_size]))
     
    hidden_weights = tf.Variable(tf.truncated_normal([batch_size, hidden_layers_size]))
    hidden_biases = tf.Variable(tf.truncated_normal([hidden_layers_size]))
    
    output_weights = tf.Variable(tf.truncated_normal([hidden_layers_size, num_labels]))
    output_biases = tf.Variable(tf.zeros([num_labels]))

    
    def network(input): 
        logit_input = tf.matmul(input, input_weights) + input_biases
        logit_hidden = tf.matmul(tf.nn.relu(logit_input), hidden_weights) + hidden_biases    
        logit_output = tf.matmul(logit_hidden, output_weights) + output_biases
        
        return logit_output + 0.01*tf.nn.l2_loss(input_weights) + 0.01*tf.nn.l2_loss(hidden_weights) \
                + 0.01*tf.nn.l2_loss(output_weights)

    # Training computation.
    predictions = network(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=predictions))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.0001).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(predictions)
    valid_prediction = tf.nn.softmax(network(tf_valid_dataset))
    test_prediction = tf.nn.softmax(network(tf_test_dataset))

num_steps = 30001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

    

Initialized
Minibatch loss at step 0: 4101.702148
Minibatch accuracy: 6.2%
Validation accuracy: 8.7%
Minibatch loss at step 500: 550.872314
Minibatch accuracy: 64.8%
Validation accuracy: 63.5%
Minibatch loss at step 1000: 423.830139
Minibatch accuracy: 60.2%
Validation accuracy: 69.5%
Minibatch loss at step 1500: 436.473724
Minibatch accuracy: 68.0%
Validation accuracy: 71.6%
Minibatch loss at step 2000: 224.951599
Minibatch accuracy: 79.7%
Validation accuracy: 72.7%
Minibatch loss at step 2500: 264.291443
Minibatch accuracy: 71.9%
Validation accuracy: 73.4%
Minibatch loss at step 3000: 214.285767
Minibatch accuracy: 75.8%
Validation accuracy: 74.2%
Minibatch loss at step 3500: 192.285629
Minibatch accuracy: 75.8%
Validation accuracy: 74.3%
Minibatch loss at step 4000: 270.868042
Minibatch accuracy: 70.3%
Validation accuracy: 74.8%
Minibatch loss at step 4500: 131.853699
Minibatch accuracy: 81.2%
Validation accuracy: 75.0%
Minibatch loss at step 5000: 155.291672
Minibatch accuracy: 80.

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [10]:
print(train_dataset.shape)
print(train_labels.shape)
train_dataset1 = train_dataset[:256, :]
train_labels1 = train_labels[:256, :]
print(train_dataset1.shape)
print(train_labels.shape)


batch_size = 128
hidden_layers_size = 1024

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(tf.truncated_normal([hidden_layers_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layers_size]))
    hidden_biases = tf.Variable(tf.truncated_normal([hidden_layers_size]))
    
    logit_hidden = tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases
    
    logit = tf.matmul(tf.nn.relu(logit_hidden), weights) + biases

    # Training computation.
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logit)) \
            + 0.01*tf.nn.l2_loss(weights) + 0.01*tf.nn.l2_loss(hidden_weights) 

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logit)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases), weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases), weights) + biases)

    
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels1.shape[0] - batch_size)

        # Generate a minibatch.
        batch_data = train_dataset1[offset:(offset + batch_size), :]
        batch_labels = train_labels1[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

(200000, 784)
(200000, 10)
(256, 784)
(200000, 10)
Initialized
Minibatch loss at step 0: 3442.722900
Minibatch accuracy: 10.9%
Validation accuracy: 28.3%
Minibatch loss at step 500: 1155.596924
Minibatch accuracy: 100.0%
Validation accuracy: 56.1%
Minibatch loss at step 1000: 424.907959
Minibatch accuracy: 100.0%
Validation accuracy: 56.2%
Minibatch loss at step 1500: 156.238693
Minibatch accuracy: 100.0%
Validation accuracy: 57.1%
Minibatch loss at step 2000: 57.463314
Minibatch accuracy: 100.0%
Validation accuracy: 61.1%
Minibatch loss at step 2500: 21.180429
Minibatch accuracy: 100.0%
Validation accuracy: 66.5%
Minibatch loss at step 3000: 7.876851
Minibatch accuracy: 100.0%
Validation accuracy: 68.7%
Test accuracy: 75.0%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

#### Normal case 

In [11]:
batch_size = 128
hidden_layers_size = 1024

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(tf.truncated_normal([hidden_layers_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layers_size]))
    hidden_biases = tf.Variable(tf.truncated_normal([hidden_layers_size]))
    
    logit_hidden = tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases
    
    logit = tf.matmul(tf.nn.dropout(tf.nn.relu(logit_hidden), 0.5), weights) + biases

    # Training computation.
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logit)) \
            + 0.01*tf.nn.l2_loss(weights) + 0.01*tf.nn.l2_loss(hidden_weights) 

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logit)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases), weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases), weights) + biases)

    
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    

Initialized
Minibatch loss at step 0: 3670.551270
Minibatch accuracy: 14.8%
Validation accuracy: 29.1%
Minibatch loss at step 500: 1154.733276
Minibatch accuracy: 78.1%
Validation accuracy: 83.6%
Minibatch loss at step 1000: 419.032898
Minibatch accuracy: 77.3%
Validation accuracy: 84.0%
Minibatch loss at step 1500: 153.859802
Minibatch accuracy: 81.2%
Validation accuracy: 84.0%
Minibatch loss at step 2000: 56.813923
Minibatch accuracy: 83.6%
Validation accuracy: 84.9%
Minibatch loss at step 2500: 21.467232
Minibatch accuracy: 76.6%
Validation accuracy: 84.6%
Minibatch loss at step 3000: 8.276829
Minibatch accuracy: 87.5%
Validation accuracy: 84.4%
Test accuracy: 90.1%


#### Overfitting case 

In [12]:
print(train_dataset.shape)
print(train_labels.shape)
train_dataset1 = train_dataset[:256, :]
train_labels1 = train_labels[:256, :]
print(train_dataset1.shape)
print(train_labels.shape)


batch_size = 128
hidden_layers_size = 1024

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(tf.truncated_normal([hidden_layers_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layers_size]))
    hidden_biases = tf.Variable(tf.truncated_normal([hidden_layers_size]))
    
    logit_hidden = tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases
    
    logit = tf.matmul(tf.nn.relu(tf.nn.dropout(logit_hidden, 0.8)), weights) + biases

    # Training computation.
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logit)) \
            + 0.01*tf.nn.l2_loss(weights) + 0.01*tf.nn.l2_loss(hidden_weights) 

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logit)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases), weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases), weights) + biases)

num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels1.shape[0] - batch_size)

        # Generate a minibatch.
        batch_data = train_dataset1[offset:(offset + batch_size), :]
        batch_labels = train_labels1[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

(200000, 784)
(200000, 10)
(256, 784)
(200000, 10)
Initialized
Minibatch loss at step 0: 3656.820801
Minibatch accuracy: 5.5%
Validation accuracy: 24.6%
Minibatch loss at step 500: 1158.013062
Minibatch accuracy: 100.0%
Validation accuracy: 63.5%
Minibatch loss at step 1000: 425.802765
Minibatch accuracy: 100.0%
Validation accuracy: 63.5%
Minibatch loss at step 1500: 156.567535
Minibatch accuracy: 100.0%
Validation accuracy: 65.3%
Minibatch loss at step 2000: 57.580704
Minibatch accuracy: 100.0%
Validation accuracy: 66.3%
Minibatch loss at step 2500: 21.215008
Minibatch accuracy: 100.0%
Validation accuracy: 68.0%
Minibatch loss at step 3000: 7.894118
Minibatch accuracy: 100.0%
Validation accuracy: 69.4%
Test accuracy: 75.8%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [13]:
import math 

batch_size = 128
hidden_layers_size = 1024
learning_starting_rate = 0.1
learning_decay_steps = 100
learning_decay_rate = 0.95
beta = 1*10^-5
keep_prob_dropouts=0.8

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(tf.truncated_normal([hidden_layers_size, num_labels]))
#     weights = tf.Variable(tf.truncated_normal([hidden_layers_size, num_labels], stddev=1 / math.sqrt(float(image_size * image_size))))
    biases = tf.Variable(tf.truncated_normal([num_labels]))
    
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layers_size]))
    hidden_biases = tf.Variable(tf.truncated_normal([hidden_layers_size]))
    
    logit_hidden = tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases
    
    logit = tf.matmul(tf.nn.dropout(tf.nn.relu(logit_hidden), keep_prob_dropouts), weights) + biases

    # Training computation.
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logit)) #\
            #+ beta*tf.nn.l2_loss(weights) + beta*tf.nn.l2_loss(hidden_weights) 

    # Optimizer.
#     optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
    
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(learning_starting_rate, global_step, learning_decay_steps, learning_decay_rate)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logit)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases), weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases), weights) + biases)


import matplotlib.pyplot as plt 
from IPython import display

train_acc_history = []
val_acc_history = []
test_acc_history = []
epochs_history = []

num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels }
        _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Offset: %d" % offset )
            print("Learning rate: %f" % lr )            
            print("Minibatch loss at step %d: %f" % (step, l))
            train_acc = accuracy(predictions, batch_labels)
            print("Minibatch accuracy: %.1f%%" % train_acc)
            val_acc = accuracy(valid_prediction.eval(), valid_labels)
            print("Validation accuracy: %.1f%%" % val_acc)
            test_acc = accuracy(test_prediction.eval(), test_labels)

            display.clear_output(wait=True)            
            epochs_history.append(step)
            train_acc_history.append(train_acc)
            val_acc_history.append(val_acc)
            test_acc_history.append(test_acc)
            
            def generate_plot(x, ys): 
                for y in ys: 
                    plt.plot(epochs_history, train_acc_history, 'r', label='Training accuracy')
                    plt.plot(epochs_history, val_acc_history, 'b', label='Validation accuracy')
                    plt.plot(epochs_history, test_acc_history, 'g', label='Test accuracy')            
                    plt.xlabel('Epochs')
                    plt.ylabel('Accuracy')
                    plt.legend()
                display.display(plt.show())
                     
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Test accuracy: 91.4%


#### Deeper network

In [15]:
batch_size = 128
hidden_layers_size = 4096

learning_starting_rate = 0.1
learning_decay_steps = 100
learning_decay_rate = 0.95
beta = 1*10^-5
keep_prob_dropouts=0.7

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    input_weights = tf.Variable(tf.truncated_normal([image_size * image_size, batch_size], mean=0.0, stddev=0.1))
    input_biases = tf.Variable(tf.truncated_normal([batch_size]))
     
    hidden_weights = tf.Variable(tf.truncated_normal([batch_size, hidden_layers_size], mean=0.0, stddev=0.1))
    hidden_biases = tf.Variable(tf.truncated_normal([hidden_layers_size]))
    
    output_weights = tf.Variable(tf.truncated_normal([hidden_layers_size, num_labels], mean=0.0, stddev=0.1))
    output_biases = tf.Variable(tf.truncated_normal([num_labels]))

    
    def network(input): 
        logit_input = tf.matmul(input, input_weights) + input_biases
        logit_hidden = tf.matmul(tf.nn.dropout(tf.nn.relu(logit_input), keep_prob_dropouts), hidden_weights) + hidden_biases    
        logit_output = tf.matmul(tf.nn.relu(logit_hidden), output_weights) + output_biases
        
        return logit_output #+ 0.01*tf.nn.l2_loss(input_weights) + 0.01*tf.nn.l2_loss(hidden_weights) \
                #+ 0.01*tf.nn.l2_loss(output_weights)

    # Training computation.
    predictions = network(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=predictions))

    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(learning_starting_rate, global_step, learning_decay_steps, learning_decay_rate)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)


    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(predictions)
    valid_prediction = tf.nn.softmax(network(tf_valid_dataset))
    test_prediction = tf.nn.softmax(network(tf_test_dataset))


import matplotlib.pyplot as plt 
from IPython import display

train_acc_history = []
val_acc_history = []
test_acc_history = []
epochs_history = []
    
num_steps = 6001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Learning rate: %f" % lr )       
            print("Minibatch loss at step %d: %f" % (step, l))
            train_acc = accuracy(predictions, batch_labels)
            print("Minibatch accuracy: %.1f%%" % train_acc)
            val_acc = accuracy(valid_prediction.eval(), valid_labels)
            print("Validation accuracy: %.1f%%" % val_acc)
            test_acc = accuracy(test_prediction.eval(), test_labels)
            
            
            #display.clear_output(wait=True)            
            epochs_history.append(step)
            train_acc_history.append(train_acc)
            val_acc_history.append(val_acc)
            test_acc_history.append(test_acc)
            

#             plt.plot(epochs_history, train_acc_history, 'r', label='Training accuracy')
#             plt.plot(epochs_history, val_acc_history, 'b', label='Validation accuracy')
#             plt.plot(epochs_history, test_acc_history, 'g', label='Test accuracy')            
#             plt.xlabel('Epochs')
#             plt.ylabel('Accuracy')
#             plt.legend()
            
#             plt.show()
            #display.display(plt.show())

    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

    

Initialized
Learning rate: 0.100000
Minibatch loss at step 0: 7.677100
Minibatch accuracy: 11.7%
Validation accuracy: 17.2%
Learning rate: 0.077378
Minibatch loss at step 500: 0.827655
Minibatch accuracy: 82.0%
Validation accuracy: 80.1%
Learning rate: 0.059874
Minibatch loss at step 1000: 0.657484
Minibatch accuracy: 78.9%
Validation accuracy: 83.1%
Learning rate: 0.046329
Minibatch loss at step 1500: 0.775079
Minibatch accuracy: 76.6%
Validation accuracy: 82.9%
Learning rate: 0.035849
Minibatch loss at step 2000: 0.472462
Minibatch accuracy: 85.9%
Validation accuracy: 84.0%
Learning rate: 0.027739
Minibatch loss at step 2500: 0.519350
Minibatch accuracy: 85.2%
Validation accuracy: 84.1%
Learning rate: 0.021464
Minibatch loss at step 3000: 0.491791
Minibatch accuracy: 86.7%
Validation accuracy: 84.9%
Learning rate: 0.016608
Minibatch loss at step 3500: 0.438713
Minibatch accuracy: 87.5%
Validation accuracy: 84.9%
Learning rate: 0.012851
Minibatch loss at step 4000: 0.543776
Minibatch 